In [1]:
import os
import sys
import json
import imgaug
import argparse
import cv2 as cv
from math import pi
import pandas as pd
import numpy as np
import skimage.draw
from fnmatch import fnmatch
from itertools import permutations
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [2]:
# Root directory of the project
ROOT_DIR = os.path.abspath("C:\\Users\\nidhimh\\Documents\\Mask_RCNN-master")

In [3]:
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.config import Config
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)


C:\Users\nidhimh\Anaconda3\envs\MaskRCNN\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nidhimh\Anaconda3\envs\MaskRCNN\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nidhimh\Anaconda3\envs\MaskRCNN\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nidhimh\Anaconda3\envs\MaskRCNN\lib\site-packages

In [4]:
class FDDBConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "FDDB"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + face

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
config = FDDBConfig()
config.display()



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [13]:
class FDDBDataset(utils.Dataset):

    def load_FDDB(self, dataset_dir, subset):
        """
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("face", 1, "face")
        
        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        data = pd.read_csv('dataforfold2.csv', index_col=0)
        
        unique = list(dict.fromkeys(list(data['imgPath'])))
        for a in unique:
            sub = data[data['imgPath'] == a]
            sub = sub.iloc[:, 1:]
            l = sub.values.tolist()
            ellipses = {'imgPath': a, 'parameters': l}
            a += '.jpg'
            filename = os.path.basename(a[15:])
            image_path = os.path.join('C:\\Users\\nidhimh\\Documents\\MaskRCNN\\dataset\\val', filename)

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "face",
                image_id=filename,  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=ellipses,
                intances = int(len(ellipses['parameters']))
            )
                
    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "face":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)
            
    def load_mask(self, image_id):
        """Generate instance masks for an image.
        Returns:
        masks: A bool array of shape [height, width, instance count] with
        one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        info = self.image_info[image_id]
        ellipse = info["polygons"]
        
        # Convert polygons to a bitmap mask of shape
        mask = np.zeros([info["height"], info["width"], info["instances"]],
                dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            rr, cc = skimage.draw.ellipse(p['center_y'], p['center_x'], p['rad_y'], p['rad_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

In [11]:
dataset_train = FDDBDataset()
dataset_train.load_FDDB('C:\\Users\\nidhimh\\Documents\\MaskRCNN\\dataset', "train")
dataset_train.prepare()

In [14]:
dataset_val = FDDBDataset()
dataset_val.load_FDDB('C:\\Users\\nidhimh\\Documents\\MaskRCNN\\dataset', "val")
dataset_val.prepare()

In [12]:
model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

In [16]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: C:\Users\nidhimh\Documents\Mask_RCNN-master\logs\fddb20210703T1951\mask_rcnn_fddb_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistr

C:\Users\nidhimh\Anaconda3\envs\MaskRCNN\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
ERROR:root:Error processing image {'id': 'img_407.jpg', 'source': 'face', 'path': 'C:\\Users\\nidhimh\\Documents\\MaskRCNN\\dataset\\train\\img_407.jpg', 'width': 449, 'height': 377, 'polygons': {'imgPath': '2002/08/04/big/img_407', 'parameters': [[33.528889, 23.135402, -1.46258, 51.156104, 173.184995], [30.909747, 21.983037, 1.551574, 8.91358, 65.935022], [31.774451, 18.555769, 1.492074, 50.80299, 4.443433], [85.67369, 55.936901, -1.343675, 167.317165, 104.467609], [41.4311, 30.154598, -1.51822, 360.953151, 328.984383], [37.144461, 27.380038, -1.483926, 421.885057, 176.003284]]}, 'intances': 6}
Traceback (most recent call last):
  File "C:\Users\nidhimh\Documents\Mask_RCNN-master\mrcnn\mod

Epoch 1/1


KeyError: 'instances'

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=2, 
            layers="all")

In [ ]:

Starting at epoch 0. LR=0.001

Checkpoint Path: C:\Users\nidhimh\Documents\Mask_RCNN-master\logs\fddb20210703T1951\mask_rcnn_fddb_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_mask_deconv      (TimeDistributed)
mrcnn_class_logits     (TimeDistributed)
mrcnn_mask             (TimeDistributed)
WARNING:tensorflow:From C:\Users\nidhimh\Anaconda3\envs\MaskRCNN\lib\site-packages\tensorflow\python\ops\math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
C:\Users\nidhimh\Anaconda3\envs\MaskRCNN\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
ERROR:root:Error processing image {'id': 'img_407.jpg', 'source': 'face', 'path': 'C:\\Users\\nidhimh\\Documents\\MaskRCNN\\dataset\\train\\img_407.jpg', 'width': 449, 'height': 377, 'polygons': {'imgPath': '2002/08/04/big/img_407', 'parameters': [[33.528889, 23.135402, -1.46258, 51.156104, 173.184995], [30.909747, 21.983037, 1.551574, 8.91358, 65.935022], [31.774451, 18.555769, 1.492074, 50.80299, 4.443433], [85.67369, 55.936901, -1.343675, 167.317165, 104.467609], [41.4311, 30.154598, -1.51822, 360.953151, 328.984383], [37.144461, 27.380038, -1.483926, 421.885057, 176.003284]]}, 'intances': 6}
Traceback (most recent call last):
  File "C:\Users\nidhimh\Documents\Mask_RCNN-master\mrcnn\model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "C:\Users\nidhimh\Documents\Mask_RCNN-master\mrcnn\model.py", line 1212, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "<ipython-input-10-348863a1dc59>", line 62, in load_mask
    mask = np.zeros([info["height"], info["width"], info["instances"]],
KeyError: 'instances'
ERROR:root:Error processing image {'id': 'img_301.jpg', 'source': 'face', 'path': 'C:\\Users\\nidhimh\\Documents\\MaskRCNN\\dataset\\train\\img_301.jpg', 'width': 450, 'height': 440, 'polygons': {'imgPath': '2002/08/13/big/img_301', 'parameters': [[95.889242, 63.425789, -1.315467, 218.609878, 121.516098]]}, 'intances': 1}
Traceback (most recent call last):
  File "C:\Users\nidhimh\Documents\Mask_RCNN-master\mrcnn\model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "C:\Users\nidhimh\Documents\Mask_RCNN-master\mrcnn\model.py", line 1212, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "<ipython-input-10-348863a1dc59>", line 62, in load_mask
    mask = np.zeros([info["height"], info["width"], info["instances"]],
KeyError: 'instances'
ERROR:root:Error processing image {'id': 'img_258.jpg', 'source': 'face', 'path': 'C:\\Users\\nidhimh\\Documents\\MaskRCNN\\dataset\\train\\img_258.jpg', 'width': 450, 'height': 448, 'polygons': {'imgPath': '2002/08/11/big/img_258', 'parameters': [[57.807612, 35.558183, -1.558006, 195.592969, 106.783771], [18.1515, 11.592792, -1.537889, 275.802184, 420.729368], [20.470656, 13.565436, 1.406736, 350.376873, 417.56531], [17.042057, 11.369755, -1.554118, 293.66, 368.0], [15.223555, 11.212343, -1.560541, 28.701937, 263.132637], [13.2708, 10.3214, 1.561045, 360.069796, 127.465736]]}, 'intances': 6}
Traceback (most recent call last):
  File "C:\Users\nidhimh\Documents\Mask_RCNN-master\mrcnn\model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "C:\Users\nidhimh\Documents\Mask_RCNN-master\mrcnn\model.py", line 1212, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "<ipython-input-10-348863a1dc59>", line 62, in load_mask
    mask = np.zeros([info["height"], info["width"], info["instances"]],
KeyError: 'instances'
ERROR:root:Error processing image {'id': 'img_1347.jpg', 'source': 'face', 'path': 'C:\\Users\\nidhimh\\Documents\\MaskRCNN\\dataset\\train\\img_1347.jpg', 'width': 358, 'height': 450, 'polygons': {'imgPath': '2002/08/01/big/img_1347', 'parameters': [[105.251752, 67.716508, -1.567968, 196.57136, 165.268997]]}, 'intances': 1}
Traceback (most recent call last):
  File "C:\Users\nidhimh\Documents\Mask_RCNN-master\mrcnn\model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "C:\Users\nidhimh\Documents\Mask_RCNN-master\mrcnn\model.py", line 1212, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "<ipython-input-10-348863a1dc59>", line 62, in load_mask
    mask = np.zeros([info["height"], info["width"], info["instances"]],
KeyError: 'instances'
ERROR:root:Error processing image {'id': 'img_961.jpg', 'source': 'face', 'path': 'C:\\Users\\nidhimh\\Documents\\MaskRCNN\\dataset\\train\\img_961.jpg', 'width': 450, 'height': 327, 'polygons': {'imgPath': '2002/07/31/big/img_961', 'parameters': [[68.722452, 47.448581, 1.540624, 135.416382, 95.183653]]}, 'intances': 1}
Traceback (most recent call last):
  File "C:\Users\nidhimh\Documents\Mask_RCNN-master\mrcnn\model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "C:\Users\nidhimh\Documents\Mask_RCNN-master\mrcnn\model.py", line 1212, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "<ipython-input-10-348863a1dc59>", line 62, in load_mask
    mask = np.zeros([info["height"], info["width"], info["instances"]],
KeyError: 'instances'
ERROR:root:Error processing image {'id': 'img_533.jpg', 'source': 'face', 'path': 'C:\\Users\\nidhimh\\Documents\\MaskRCNN\\dataset\\train\\img_533.jpg', 'width': 269, 'height': 410, 'polygons': {'imgPath': '2002/08/02/big/img_533', 'parameters': [[126.707387, 75.551435, -1.554628, 153.807558, 162.914902]]}, 'intances': 1}
Traceback (most recent call last):
  File "C:\Users\nidhimh\Documents\Mask_RCNN-master\mrcnn\model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "C:\Users\nidhimh\Documents\Mask_RCNN-master\mrcnn\model.py", line 1212, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "<ipython-input-10-348863a1dc59>", line 62, in load_mask
    mask = np.zeros([info["height"], info["width"], info["instances"]],
KeyError: 'instances'
Epoch 1/1
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
<ipython-input-16-83fb3ae74319> in <module>
      6             learning_rate=config.LEARNING_RATE,
      7             epochs=1,
----> 8             layers='heads')

~\Documents\Mask_RCNN-master\mrcnn\model.py in train(self, train_dataset, val_dataset, learning_rate, epochs, layers, augmentation, custom_callbacks, no_augmentation_sources)
   2372             max_queue_size=100,
   2373             workers=workers,
-> 2374             use_multiprocessing=True,
   2375         )
   2376         self.epoch = max(self.epoch, epochs)

~\Anaconda3\envs\MaskRCNN\lib\site-packages\keras\legacy\interfaces.py in wrapper(*args, **kwargs)
     89                 warnings.warn('Update your `' + object_name +
     90                               '` call to the Keras 2 API: ' + signature, stacklevel=2)
---> 91             return func(*args, **kwargs)
     92         wrapper._original_function = func
     93         return wrapper

~\Anaconda3\envs\MaskRCNN\lib\site-packages\keras\engine\training.py in fit_generator(self, generator, steps_per_epoch, epochs, verbose, callbacks, validation_data, validation_steps, class_weight, max_queue_size, workers, use_multiprocessing, shuffle, initial_epoch)
   2192                 batch_index = 0
   2193                 while steps_done < steps_per_epoch:
-> 2194                     generator_output = next(output_generator)
   2195 
   2196                     if not hasattr(generator_output, '__len__'):

~\Documents\Mask_RCNN-master\mrcnn\model.py in data_generator(dataset, config, shuffle, augment, augmentation, random_rois, batch_size, detection_targets, no_augmentation_sources)
   1707                     load_image_gt(dataset, config, image_id, augment=augment,
   1708                                 augmentation=augmentation,
-> 1709                                 use_mini_mask=config.USE_MINI_MASK)
   1710 
   1711             # Skip images that have no instances. This can happen in cases

~\Documents\Mask_RCNN-master\mrcnn\model.py in load_image_gt(dataset, config, image_id, augment, augmentation, use_mini_mask)
   1210     # Load image and mask
   1211     image = dataset.load_image(image_id)
-> 1212     mask, class_ids = dataset.load_mask(image_id)
   1213     original_shape = image.shape
   1214     image, window, scale, padding, crop = utils.resize_image(

<ipython-input-10-348863a1dc59> in load_mask(self, image_id)
     60 
     61         # Convert polygons to a bitmap mask of shape
---> 62         mask = np.zeros([info["height"], info["width"], info["instances"]],
     63                 dtype=np.uint8)
     64         for i, p in enumerate(info["polygons"]):

KeyError: 'instances'